In [ ]:
%%capture
!pip install umap-learn transformers pymorphy3

In [ ]:
import torch
import pandas as pd
import numpy as np
import torch
import datetime
import pymorphy3

from tqdm import tqdm
from torch.utils.data import DataLoader
from dateutil import parser
# from transformers import AutoModel, AutoTokenizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!gdown 11SQswlJWdxCiWzO0rG1ttNT4CePLRYIM

Downloading...
From: https://drive.google.com/uc?id=11SQswlJWdxCiWzO0rG1ttNT4CePLRYIM
To: /content/results_sravni (2).csv
100% 38.6M/38.6M [00:00<00:00, 60.5MB/s]


In [ ]:
def preprocess_triplets(res_df, date_df, triplets_columns="triplets", date_column="timestamp", morph=True):
  if morph: morph_parser = pymorphy3.MorphAnalyzer()
  triplets = res_df[triplets_columns].values.tolist()

  dates = date_df[date_column].values.tolist()
  all_triplets = []
  for i, (text, label_ix, label) in enumerate(triplets):
      all_triplets.extend([[text] + list(item) + [list(item_ix)[0]] + [dates[i]] for item_ix, item in zip(eval(label_ix), eval(label))])

  data = []
  if not morph:
     for triplet in tqdm(all_triplets):
        txt, at, ot, sp, (at_ix), ts = triplet
        at = at.lower()
        ot = ot.lower()
        data.append([ts, txt, at, at_ix, ot, sp, (at, ot, sp), (at, ot)])
  else:
    for triplet in tqdm(all_triplets):
        txt, at, ot, sp, (at_ix), ts = triplet
        at = morph_parser.parse(at.lower())[0].normal_form
        ot = ot.lower()
        data.append([ts, txt, at, at_ix, ot, sp, (at, ot, sp), (at, ot)])
  data = pd.DataFrame(data)
  data.columns = ['date', "text", 'aspect', "aspect_ix", 'opinion', 'sentiment', 'triplet', 'aspect_opinion']

  data["timestamp"] = data["date"].apply(lambda x: parser.isoparse(x).timestamp())

  return data

In [ ]:
df = pd.read_csv("/content/results_sravni (2).csv")
data = preprocess_triplets(df, df, ["text", "pred", "pred_text"], "time")

100%|██████████| 48214/48214 [00:25<00:00, 1922.19it/s]


In [ ]:
data.iloc[25, [1, 2]].text, data.iloc[25, [1, 2]].aspect

('Опыт с банком самый положительный . Несмотря на то что сама сотрудник другого банка , я амбасадор Тинькоф . Лучший банк !',
 'банк')

In [ ]:
BASE_PROMPT = """User
Ты помогаешь размечать датасет сотруднику Тинькофф банка.

### 1 Задача:

Мы хотим классифицировать аспекты отзывов на 2 категории

Перечисляю категории здесь: "Информативно",  "Неинформативно".
!!ОТНОСИ АСПЕКТ К КАТЕГОРИИ "Неинформативные" ТОЛЬКО ЕСЛИ ЭТОТ АСПЕКТ КОНКРЕТНО В ЭТОМ ОТЗЫВЕ ВЫРАЖАЕТ ЧТО-ТО ОЧЕНЬ ОБОБЩЁННОЕ!!


#Задача 2:

Мы хотим разделить аспекты отзывов из категории "Информативно" на несколько подкатегорий

!!!ВНИМАНИЕ, ЕСЛИ ОТЗЫВ ОТНОСИТСЯ К КАТЕГОРИИ "Неинформативно" ТО ПРАВИЛЬНОЙ КАТЕГОРИЕЙ БУДЕТ ПУСТАЯ СТРОКА!!!
Перечисляю подкатегории здесь: "Обслуживание",  "Общее впечатление о банке",  "Интерфейс", "Банковские условия", "Функционал", "Конкуренты".
!ЕСЛИ АСПЕКТ НЕ ОТНОСТИТСЯ НЕ К ОДНОЙ ИЗ ПОДКАТЕГОРИИ ОТНЕСИ ОТЗЫВ К ПОДКАТЕГОРИИ "Другое"!


За каждый правильно выполненный запрос, я дам тебе 1000 долларов. Четырежды проверь себя


Привожу несколько примеров правильной разметки:

Аспект: "Техподдержка"
Отзыв: "Давно была мысль оставить отзыв об этом банке ) Обслуживаюсь в Тинькофф банке уже более 4 лет , а в Тинькофф Бизнес уже более 2 лет , лучшие условия обслуживания , лучший кешбек ! Что касается интернет эквайринга для бизнеса - аналогов у популярных банков с комиссией выгоднее не нашл . Если первый опыт в открытии ИП - Тинькофф максимально все сделает за вас ) Много отличных сервисов таких как бухгалтерия , инвестиции , мобайл , которые интуитивно понятны даже новичку ! Конечно , чтобы получить больший профит от продуктов - придтся разобраться в тонкостях тарифов , в том числе и как грамотно получать кешбек и совершать покупки выгоднее ! Техподдержка во всех сервисах оперативная ( хотя раньше есть ощущения , что могли отвечать долго ) , но сейчас проблем с этим не наблюдаю , вс довольно понятно ! Персональный менеджер , к которому модно обратится по большинству вопросов хоть в чате , хоть позвонить - это очень удобно ! Отдельное спасибо Нине Ш . за оперативные ответы ) Тинькофф , надеюсь дождусь бухгалтерию УСН с сотрудниками , чтобы все работало в одной экосистеме ."
Твой ответ: "Информативно, Обслуживание"

Аспект: "Банк"
Отзыв: "Оплатила на Кубе экскурсию дебетовой картой Тинькофф Блэк . По данной покупке кэшбэк не начислили , т к не по всем покупкам это происходит , есть какие - то коды , по которым и в РФ не начисляют . По приезде увидела и позвонила в банк . На следующий день по обращению произвели начисление . Мелочь , 58 рублей , но очень приятно . Очень благодарна . Из таких мелочей складывается впечатление . Самый лучший банк - Тинькофф !"
Твой ответ: "Неинформативно"

Аспект: "техподдержка"
Отзыв: "техподдержке спасибо!"
Твой ответ: "Неинформативно"


!!!ОЧЕНЬ ВАЖНО: ОТВЕТ ПО ДОЛЖЕН БЫТЬ ТОЧНО ТАКИМ ЖЕ КАК В ПРИМЕРАХ!!!

"""

def get_prompt(text, aspect):
  prompt = BASE_PROMPT + "\nаспект: " + aspect + "\nотзыв: " + text
  return prompt

In [ ]:
prompts = data[["text", "aspect"]].apply(lambda x: get_prompt(x[0], x[1]), axis=1)
result = data[["text", "aspect"]]
result["prompt"] = prompts

<ipython-input-23-565ee008e81a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["prompt"] = prompts


In [ ]:
result[:1000]["prompt"][127]

'User\nТы помогаешь размечать датасет сотруднику Тинькофф банка.\n\n### 1 Задача:\n\nМы хотим классифицировать аспекты отзывов на 2 категории\n\nПеречисляю категории здесь: "Информативно",  "Неинформативно".\n!!ОТНОСИ АСПЕКТ К КАТЕГОРИИ "Неинформативные" ТОЛЬКО ЕСЛИ ЭТОТ АСПЕКТ КОНКРЕТНО В ЭТОМ ОТЗЫВЕ ВЫРАЖАЕТ ЧТО-ТО ОЧЕНЬ ОБОБЩЁННОЕ!!\n\n\n#Задача 2:\n\nМы хотим разделить аспекты отзывов из категории "Информативно" на несколько подкатегорий\n\n!!!ВНИМАНИЕ, ЕСЛИ ОТЗЫВ ОТНОСИТСЯ К КАТЕГОРИИ "Неинформативно" ТО ПРАВИЛЬНОЙ КАТЕГОРИЕЙ БУДЕТ ПУСТАЯ СТРОКА!!!\nПеречисляю подкатегории здесь: "Обслуживание",  "Общее впечатление о банке",  "Интерфейс", "Банковские условия", "Функционал", "Конкуренты".\n!ЕСЛИ АСПЕКТ НЕ ОТНОСТИТСЯ НЕ К ОДНОЙ ИЗ ПОДКАТЕГОРИИ ОТНЕСИ ОТЗЫВ К ПОДКАТЕГОРИИ "Другое"!\n\n\nЗа каждый правильно выполненный запрос, я дам тебе 1000 долларов. Четырежды проверь себя\n\n\nПривожу несколько примеров правильной разметки:\n\nАспект: "Техподдержка"\nОтзыв: "Давно была мысль остави

In [ ]:
result[:1000].to_csv("1k_prompts_combined_fixed_final.csv", index=False)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/prompts.csv")

In [ ]:
df.iloc[:10000].iloc[0]

text      Менеджер Ирина ( внутренний номер 6907919 . ) ...
aspect                                             менеджер
prompt    Ты помогаешь размечать датасет сотруднику Тинь...
Name: 0, dtype: object

In [ ]:
result.to_csv("prompts_cats.csv", index=False)

In [ ]:
from collections import Counter

Counter(data["aspect"]).most_common(15)

[('банк', 10621),
 ('приложение', 5068),
 ('обслуживание', 2291),
 ('сотрудник', 2224),
 ('поддержка', 1613),
 ('условие', 1340),
 ('сервис', 1273),
 ('карта', 1182),
 ('кэшбэк', 1031),
 ('кэшбек', 952),
 ('работа', 790),
 ('интерфейс', 726),
 ('кешбек', 683),
 ('менеджер', 680),
 ('решение', 602)]

In [ ]:
data[data.aspect == "банк"].text.iloc[0]

'Оплатила на Кубе экскурсию дебетовой картой Тинькофф Блэк . По данной покупке кэшбэк не начислили , т к не по всем покупкам это происходит , есть какие - то коды , по которым и в РФ не начисляют . По приезде увидела и позвонила в банк . На следующий день по обращению произвели начисление . Мелочь , 58 рублей , но очень приятно . Очень благодарна . Из таких мелочей складывается впечатление . Самый лучший банк - Тинькофф !'

In [ ]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "DeepPavlov/rubert-base-cased-sentence"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [ ]:
aspects_opinions = list(set(data["aspect_opinion"]))
model = model.to(device).eval()
with torch.no_grad():
    embeddings = {at_on : model(**tokenizer([at_on[0] + " " + at_on[1]], return_tensors="pt").to(device)).pooler_output[0] for at_on in aspects_opinions}

In [ ]:
def axis_bounds(embedding):
    left, right = embedding.T[0].min(), embedding.T[0].max()
    bottom, top = embedding.T[1].min(), embedding.T[1].max()
    adj_h, adj_v = (right - left) * 0.1, (top - bottom) * 0.1
    return [left - adj_h, right + adj_h, bottom - adj_v, top + adj_v]

In [ ]:
import sklearn.datasets
import umap
import umap.utils as utils
import umap.aligned_umap
import matplotlib.pyplot as plt

constant_dict = {i:i for i in range(1181)}
constant_relations = [constant_dict for i in range(9)]

neighbors_mapper = umap.AlignedUMAP(
    n_neighbors=[2,3,4,5,7,11,16,22,29,37,45,54,62,71,85],
    min_dist=[0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45],
    alignment_window_size=2,
    alignment_regularisation=1e-3,
).fit(
    [[_.cpu().detach() for _ in list(embeddings.values())] for i in range(10)], relations=constant_relations
)

/usr/local/lib/python3.10/dist-packages/umap/spectral.py:550: UserWarning: Spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(
/usr/local/lib/python3.10/dist-packages/umap/spectral.py:550: UserWarning: Spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(
/usr/local/lib/python3.10/dist-packages/umap/spectral.py:550: UserWarning: Spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(
/usr/local/lib/python3.10/dist-packages/umap/spectral.py:550: UserWarning: Spectral initialisation failed! The eigenvector solver
failed. This is likely d

In [ ]:
from sklearn.cluster import DBSCAN

up = umap.UMAP(n_components=6)
cl = DBSCAN(n_jobs=-1).fit(up.fit_transform([_.cpu().detach() for _ in list(embeddings.values())]))

In [ ]:

fig, axs = plt.subplots(5,2, figsize=(10, 20))
ax_bound = axis_bounds(np.vstack(neighbors_mapper.embeddings_))
for i, ax in enumerate(axs.flatten()):
    ax.scatter(*neighbors_mapper.embeddings_[i].T, s=1, c=cl.labels_, cmap="Spectral")
    ax.axis(ax_bound)
    ax.set(xticks=[], yticks=[])
plt.tight_layout()